In [128]:
#install libraries need
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [129]:
# import Libraries
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from google.colab import files


In [130]:
# read links from text 
def read_links(path):
  with open(path , 'r') as links:
    link = links.readlines()
  return   link

In [131]:
# get data form website
def fetch(link):
  print('HTTP GET Request to URL %s' %link , end ='')
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

  url = link

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)

  driver.get(url)
  return driver

In [132]:
# get table header
def get_theader(table) : 
  header =[]
  try:
    thead = table.find_all('thead')[0]

    #remove tag with display: none
    for hidden in thead.find_all(style=re.compile(r'display:\s*none')):
      hidden.decompose()
      print(hidden.decompose)

    for i in thead.find_all('th'):
      if len(i.find_all('strong')) != 0  :
        st_tag = i.find('strong')
        header.append(st_tag.get_text().replace(u'\xa0', u' ').replace(' ','').replace('\n','').replace('/',''))
      
      elif len(i.find_all('span')) != 0:
        sp_tag = i.find('span')
        header.append(sp_tag.get_text().replace(u'\xa0', u' ').replace(' ','').replace('\n','').replace('/',''))

      else:
        header.append(i.get_text().replace(u'\xa0', u' ').replace(' ','').replace('\n','').replace('/',''))
  except :
    pass

  print('header before : ' , len(header))
  
  return header

In [133]:
def get_tbody(table,header):
  header_list = get_theader(table)
  #df = pd.DataFrame(columns = header_list)
  t_data = []
  try:

    tbody = table.find('tbody')  
    #for tbody in table.find_all('tbody'):
        #remove tag with display: none
    for hidden in tbody.find_all(style=re.compile(r'display:\s*none')):
      hidden.decompose()
      
    rows = tbody.find_all('tr')
    for row in rows:
      row_data =[]
      image  = row.find_all('td')[0].img['data-content']
      row_data.append(image)
  
      datasheet_tag = row.find_all('td')[1]
      if not datasheet_tag.find('a'):
          datasheet = 'None'
          row_data.append(datasheet) 
      else:
          datasheet = datasheet_tag.find('a')['href']
          row_data.append(datasheet) 

      product = row.find_all('td')[2].a.text.strip()
      product_desc = row.find_all('td')[3].text.strip()
      stock = row.find_all('td')[4].span.text.strip()
      mpq_price = row.find_all('td')[5].text.strip()
      Qty = row.find_all('td')[6].text.strip()
      manufacture = row.find_all('td')[7].text.strip()
      
      row_data.extend([product, product_desc, stock, mpq_price, Qty, manufacture])
    
      for td in row.find_all('td')[8: ]:
          row_data.append(td.get_text().strip() )

      #handle case of len(row_data) < len(header)
      if len(row_data) < len(header):
        diff = len(header) - len(row_data)
        for i in range(0,diff):
          row_data.append('None')
      
      t_data.append(row_data)
  except :
    pass
        #length = len(df)
        #df.loc[length] = row_data

  return t_data

In [134]:
#parsing Website's data
def parse(driver):
  try:  
    soup = BeautifulSoup(driver.page_source,'lxml')
    table = soup.find('table' , {'id' : 'prdTable'})
    
    header_list = get_theader(table)
    t_data_list = get_tbody(table , header_list) 
    print('t_data_list of t_body shape : ' , len(t_data_list))
  
    df = pd.DataFrame(t_data_list , columns = header_list)
    data = pd.DataFrame()
    data = data.append(df , ignore_index =True) 

    print('data  shape : ' , data.shape)

    # get data from next page
    condition = True
    while condition :
      pg = soup.find('ul' , {'class' : 'pagination'})
      active_pg = pg.find('li' , {'class' : 'active'})
      if not active_pg.findNextSibling('li') :
        condition = False
      else: 
        next_url = active_pg.findNextSibling('li').a.get('href')
        url = 'https://www.ozdisan.com'+next_url

        driver = fetch(url)

        soup = BeautifulSoup(driver.page_source,'lxml')
        table = soup.find('table' , {'id' : 'prdTable'})

        header_list = get_theader(table)
        t_data_list = get_tbody(table , header_list)
        df = pd.DataFrame(t_data_list , columns = header_list)
        data = data.append(df , ignore_index =True) 
        print('data  shape : ' , data.shape)
  except :
    pass
  return data

In [135]:
#save File
def save_file(df , counter):
  print('saved file shape is : ', df.shape)
  df.to_excel(f'project/{counter}.xlsx' , index= False)
  print(f'project/{counter}.xlsx')
   


In [136]:
# run code to execute all script
def run ():
  counter = 702
  for link in read_links('/content/15.txt'):
    driver = fetch(link)
    df = parse(driver)
    save_file(df, counter)
    counter = counter +1
    time.sleep(10)
    


In [137]:
# remove old project folder
!rm -rf project
# remove old project.zip file
!rm project.zip

#create project folder
!mkdir project

In [138]:
#run script
run()

HTTP GET Request to URL https://www.ozdisan.com/p/antistatic-cleaning-products-10470?page=&sayfaAdedi=250&inStock=true
<bound method Tag.decompose of <None></None>>
<bound method Tag.decompose of <None></None>>
header before :  13
header before :  13
t_data_list of t_body shape :  4
data  shape :  (4, 13)
saved file shape is :  (4, 13)
project/702.xlsx
HTTP GET Request to URL https://www.ozdisan.com/p/antistatic-ic-vacuum-pens-10473?page=&sayfaAdedi=250&inStock=true
header before :  0
header before :  0
t_data_list of t_body shape :  0
data  shape :  (0, 0)
saved file shape is :  (0, 0)
project/703.xlsx
HTTP GET Request to URL https://www.ozdisan.com/p/antistatic-repairing-tools-10471?page=&sayfaAdedi=250&inStock=true
<bound method Tag.decompose of <None></None>>
<bound method Tag.decompose of <None></None>>
header before :  13
header before :  13
t_data_list of t_body shape :  1
data  shape :  (1, 13)
saved file shape is :  (1, 13)
project/704.xlsx
HTTP GET Request to URL https://www.

In [139]:
# Create zip Folder with saved files
!zip -r /content/project.zip /content/project

#download zip folder
files.download('/content/project.zip')

  adding: content/project/ (stored 0%)
  adding: content/project/726.xlsx (deflated 9%)
  adding: content/project/722.xlsx (deflated 5%)
  adding: content/project/727.xlsx (deflated 9%)
  adding: content/project/743.xlsx (deflated 10%)
  adding: content/project/764.xlsx (deflated 10%)
  adding: content/project/759.xlsx (deflated 10%)
  adding: content/project/733.xlsx (deflated 9%)
  adding: content/project/755.xlsx (deflated 10%)
  adding: content/project/714.xlsx (deflated 10%)
  adding: content/project/744.xlsx (deflated 10%)
  adding: content/project/742.xlsx (deflated 8%)
  adding: content/project/741.xlsx (deflated 10%)
  adding: content/project/718.xlsx (deflated 9%)
  adding: content/project/716.xlsx (deflated 10%)
  adding: content/project/754.xlsx (deflated 10%)
  adding: content/project/746.xlsx (deflated 12%)
  adding: content/project/705.xlsx (deflated 8%)
  adding: content/project/737.xlsx (deflated 12%)
  adding: content/project/725.xlsx (deflated 12%)
  adding: content/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>